In [1]:
import logging
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M_')
loggerFilename = "{}reqBuilder.log".format(timestamp)
outputFilename = "{}requests.csv".format(timestamp)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
# create a file handler
handler = logging.FileHandler(loggerFilename)
handler.setLevel(logging.INFO)
# create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(handler)

# Sample bounding box taken from GAIA - located somewhere in the CBD, These are in long and lat (Tools->Map Region)
#minx = 145.031036717514
#maxx = 145.03631986817
#miny = -37.9016436199178
#maxy = -37.8963604692623

minx = 145.282742318613
maxx = 145.288193690464
miny = -37.8865223988944
maxy = -37.8830873778363

# The Output tiles are PNG, this is the resolution of the tile returned by the query
tileWidth = 1024
tileHeight = 768

# WMS Protocol Version
wmsVersion = '1.3.0'

# Base Storm address
stormAddress = 'http://127.0.0.1/spatialSTORM'

modelVersion = 'v8_2_1'

# These are the layers to include in the query - A single PNG with all these layers will be returned
layers = 'NetworkNodes,SupportCabinets,NetworkCables,SupportDucts,SupportPillars,SupportPits,SupportStrands,NetworkEquipments,SupportPoles' # 'SupportPoles,Copper' # comma separated

# Increases the bounding box size - When the bounding box difference in x and y is calculated, the box will be approx 580m in both long and lat. This is a quick adjustment
bbMultiplier = 1 # Multiplier for bounding box # simple way to expand the grid size, see km(x) and km(y) changes

# Calculates the difference in X and Y, this is used in the loop later to create a grid of tiles
diffX = (maxx-minx)*bbMultiplier # Latitude Degrees
diffY = (maxy-miny)*bbMultiplier

# This is what 1 degree long/lat equates to in km's and miles
unitDegKms = 110.567 # 1 lat or long degree is 110.567 km's
unitDegMiles = 68.703 # 1 lat or long degree is 68.703 miles

# Will tack on extra wms query params to the end of the request, must start with &, each param separated by &
extraRequestParams = '' # e.g &BGCOLOR=0xFFFFFF

# General Debug information to tie against the output
logger.info("****************** Tool Started ******************".format(minx, maxx, miny, maxy))
logger.info("Bounding Box[ minx:{}, maxx:{}, miny:{}, maxy:{} ]".format(minx, maxx, miny, maxy))
logger.info("tileWidth:{}, tileHeight:{}".format(tileWidth, tileHeight))
logger.info("StormAddress:{}".format(stormAddress))
logger.info("Layers:{}".format(layers))
logger.info("ExtraParams:{}".format(extraRequestParams))
logger.info("WMSVersion:{}".format(wmsVersion))
logger.info("Multiplier: {}".format(bbMultiplier))
logger.info("Lat(x):{}, Long(y):{}".format(diffX/bbMultiplier, diffY/bbMultiplier))
logger.info("Km(x):{}, Km(y):{}".format((diffX*unitDegKms)/bbMultiplier, (diffY*unitDegKms)/bbMultiplier))
logger.info("Lat(x*multiplier):{}, Long(y*multiplier):{}".format(diffX, diffY))
logger.info("Km(x*multiplier):{}, Km(y*multiplier):{}".format(diffX*unitDegKms, diffY*unitDegKms))

# General storage variables for later calculating the grid size
count = 0
minXGrid = 0.0
minYGrid = 0.0
maxXGrid = 0.0
maxYGrid = 0.0

# Main logic:
#   Opens a file for writing,
#   Creates a grid of 100x100 tiles (10000 queries)
#   Calculates the min and max of each axis for the bounding box in each query
#   Write the query line by line
with open(outputFilename, "w") as f:
    for y in range(-50, 50):
        for x in range (-50, 50):
            xTempMin = ((x-1)*diffX+minx)
            xTempMax = (x*diffX+minx)
            yTempMin = ((y-1)*diffY+miny)
            yTempMax = (y*diffY+miny)
            #print xTempMin, xTempMax, yTempMin, yTempMax
            if count == 0:
                minXGrid = xTempMin
                minYGrid = yTempMin
            f.write("{}/spatialSUITE/{}/wms?REQUEST=GetMap&SERVICE=WMS&VERSION={}&LAYERS={}&STYLES=&FORMAT=image/png&TRANSPARENT=TRUE&CRS=EPSG:4326&BBOX={},{},{},{}&WIDTH={}&HEIGHT={}{}\n".format(stormAddress, modelVersion, wmsVersion, layers, yTempMin, xTempMin, yTempMax, xTempMax, tileWidth, tileHeight, extraRequestParams))
            count += 1
    maxXGrid = xTempMax
    maxYGrid = yTempMax

# Calculates the grid difference in degrees
diffXGrid = (maxXGrid - minXGrid)  # Latitude Degrees
diffYGrid = (maxYGrid - minYGrid)

# Outputs the grid size to file in both degrees and km's spanned
logger.info("GRID [ minx:{}, maxx:{}, miny:{}, maxy:{} ]".format(minXGrid, maxXGrid, minYGrid, maxYGrid))
logger.info("GRID Span: LatGRIDdeg(x):{}, LongGRID(y):{}".format(diffXGrid, diffYGrid))
logger.info("GRID Span: LatGRIDKm(x):{}, LongGRIDKm(y):{}".format(diffXGrid * unitDegKms, diffYGrid * unitDegKms))

# Writes the count value and filename of the output
logOut = "{} Requests written to file {}".format(count, outputFilename)
logger.info(logOut)
print(logOut)

logger.info("****************** Tool Finished ******************".format(minx, maxx, miny, maxy))



10000 Requests written to file 20191203_0202_requests.csv
